# Finite State Machine

References:
- [Context Managers and Python's with Statement](https://realpython.com/python-with-statement/)

Let $x_t$ be the stochastic process:

$$
dx_t=\mu dt+\sigma dw_t
$$

We want to model $x_t$ with trend and flat behaviors. We call a process flat, if $\mu=0$, otherwise the process is called trend. There are two scenarios for trend processes, trend up and trend down, in which casees, the model parameter $\mu$ are positive and negative repectively. So in this setting, we make the parameter $\mu$ depend on the states.

Let $S_t$ be the state process, where $S_t\in [-1,0,1]$, in other words, $S_t$ is a three states process, $-1$ corresponding to down tren, $0$ for flat and $1$ for trend up.

The fact that $\mu$ depends $S_t$, makes the law governing the process $x_t$ change over time. This kind of processes are called hidden markov processes.

In summary, we have

$$
dx_t=\mu(S_t) dt+\sigma dW_t
$$

with $S_t$ being a state process. The law governing $S_t$ is the transition probabilities, $P\{S_{t_i}=u\ |\ S_{t_{i-1}}=v\},\ u,v\in[-1,0,1]$. These probabilities are given and the are not depending on time.

We are going to develop a discrete simulator to simulate $x_t$.


In [34]:
from contextlib import contextmanager

@contextmanager
def hello_context_manager():
    print("Entering the context...")
    yield "Hello, World!"
    print("Leaving the context...")

with hello_context_manager() as hello:
    print(hello)

Entering the context...
Hello, World!
Leaving the context...


In [35]:
import numpy as np
from nptyping import NDArray, Shape, Float64

ModelParam=tuple[float,float]
StateParam=NDArray[Shape['3,3'],Float64]

In [36]:
print(StateParam)

NDArray[Shape['3, 3'], Float]


### The model

In [103]:
class hidden_markov_process():
    SV=[-1,0,1]
    def __init__(self,mp,tp):
        self.__mp=mp
        self.__tp=tp

    def transition(self,start:int, end:int)->list[float]:
        if start in self.SV and end in self.SV: return self.__tp[start+1][end+1]
        else: raise ValueError("Illegal state value")

    def diffusion(self,s:int)->tuple[float,float]:
        return self.__mp[s+1]
    
    def state_value(self): return self.SV

In [105]:
MP=[(-1.0,20.0),(0.0,30.0),(1.0,20.0)] # model parameters
TP=[[.25,.75,0.0],[.15,.7,.15],[0.0,.75,.25]] # transition probabilities
model=hidden_markov_process(MP,TP)

In [106]:
[[model.transition(start,end) for end in model.state_value()] for start in model.state_value()]

[[0.25, 0.75, 0.0], [0.15, 0.7, 0.15], [0.0, 0.75, 0.25]]

In [107]:
[model.diffusion(s) for s in model.state_value()]

[(-1.0, 20.0), (0.0, 30.0), (1.0, 20.0)]

## Simulating the state process

In [108]:
N=pow(2,13)
@contextmanager
def flat():
    print("Entering the flat state...")
    print("State transition probabilites:",tp[1])
    yield tp[1]
    print("leaving the flat state")

@contextmanager
def uptrend():
    pass

@contextmanager
def downtrend():
    pass

### Checking random number generator

In [113]:
from functools import partial
np.random.seed(0)
a=1
b=1000
# generate state
def gState(sv:int,m:hidden_markov_process,rn:int)->int:
    end=model.state_value()
    p=[m.transition(sv,x) for x in m.state_value()]
    for 
    if rn<(b-a)*p[0]: return end[0]
    elif rn<(b-a)*(p[1]+p[0]): return end[1]
    else: return end[2]

def flat_process(length:int)->map:
    with flat() as fs:
        rn = [np.random.randint(a, b) for _ in range(length)]
        return map(partial(gState,0,model),rn)


In [115]:
s=list(flat_process(N))
[len(list(filter((lambda x:x==u),s)))/N for u in [-1,0,1]]

Entering the flat state...
State transition probabilites: [0.15, 0.7, 0.15]
leaving the flat state


[0.145263671875, 0.7100830078125, 0.1446533203125]